In [1]:
import wallycore as wally

In [2]:
def buildTransaction(tx_inputs, tx_outputs):
    tx = wally.tx_init(2, 0, 1, 2) # version 2, locktime 0, 1 input, 2 outputs

    for tx_input in tx_inputs:
        wally.tx_add_input(tx, tx_input)

    for tx_output in tx_outputs:
        wally.tx_add_output(tx, tx_output)
        
    return tx
    
def buildInput(tx_input_hex, tx_input_vout, sequence):
    tx_input_byte = wally.hex_to_bytes(tx_input_hex)[::-1]
    return wally.tx_elements_input_init(tx_input_byte, tx_input_vout, sequence, None, None, None, None, None, None, None, None, None)

def buildOutput(value, asset_id, script_byte):
    confidential_tx_value_byte = wally.tx_confidential_value_from_satoshi(value)
    return wally.tx_elements_output_init(script_byte, asset_id, confidential_tx_value_byte, None, None, None)

In [3]:
ELEMENTS_UNBLINDED_TX_PREFIX = 0x01    

tx_input_hex = "53174bc15eef5b16522b5fe45cf2e147e829990dd72454ab7d2a493e220e4c36"
tx_input_vout = 0
asset_id = bytearray([ELEMENTS_UNBLINDED_TX_PREFIX]) + wally.hex_to_bytes("9e56d67dee0fbe395374572d2a389e56659380e2b58ad43081e0d3ddcab1a0f7")[::-1]
sequence = 0xffffffff

script = wally.hex_to_bytes("76a9144f844a62154987b501dccfd6c504596759e9f24888ac")
tx_inputs = [buildInput(tx_input_hex, tx_input_vout, sequence)]
tx_outputs = [buildOutput(12345, asset_id, script), buildOutput(500, asset_id, None)]

tx = buildTransaction(tx_inputs, tx_outputs)

tx_hex = wally.tx_to_hex(tx, 0)
print(tx_hex)

020000000001364c0e223e492a7dab5424d70d9929e847e1f25ce45f2b52165bef5ec14b17530000000000ffffffff0201f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e010000000000003039001976a9144f844a62154987b501dccfd6c504596759e9f24888ac01f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e0100000000000001f4000000000000


In [4]:
def signInputs(tx, private_keys, input_values, sighashes):  
    for index, (private_key, input_value, sighash) in enumerate(zip(private_keys, input_values, sighashes)):
        witness_stack = wally.tx_witness_stack_init(2)
        
        public_key  = wally.ec_public_key_from_private_key(private_key)
        prev_out_script = bytearray([0x19, wally.OP_DUP, wally.OP_HASH160, 0x14]) + wally.hash160(public_key) + bytearray([wally.OP_EQUALVERIFY, wally.OP_CHECKSIG])
        input_value = wally.tx_confidential_value_from_satoshi(input_value);
        hash_to_sign = wally.tx_get_elements_signature_hash(tx, index, prev_out_script, input_value, sighash, wally.WALLY_TX_FLAG_USE_WITNESS)
        signature = wally.ec_sig_from_bytes(private_key, hash_to_sign, wally.EC_FLAG_ECDSA | wally.EC_FLAG_GRIND_R)
        der_signature = wally.ec_sig_to_der(signature) + bytearray([wally.WALLY_SIGHASH_ALL])
        wally.tx_witness_stack_add(witness_stack, der_signature)
        
        wally.tx_witness_stack_add(witness_stack, public_key)
        script_sig = bytearray([0x16, 0x00, 0x14]) + wally.hash160(public_key)
        wally.tx_set_input_script(tx, index, script_sig)
        wally.tx_set_input_witness(tx, index, witness_stack)

In [5]:
private_key = wally.hex_to_bytes("33a61b1527bffa1a86f851a1c8d346e30f716646c128c074bfe258bb02021c79")

signInputs(tx, [private_key], [12845], [wally.WALLY_SIGHASH_ALL])

tx_hex = wally.tx_to_hex(tx, 0)
print(tx_hex)

020000000001364c0e223e492a7dab5424d70d9929e847e1f25ce45f2b52165bef5ec14b1753000000001716001412f1e8c92a3e36e87f9815b5adfbfbb57343bed2ffffffff0201f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e010000000000003039001976a9144f844a62154987b501dccfd6c504596759e9f24888ac01f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e0100000000000001f4000000000000
